# Who is hate targeted against?





### Introduction

In this project, we train a BERT model to identify the community groups against whom hate speech is usually targeted towards.

We believe that this is an important problem to solve because it helps in explaining the output of the hate speech detection models.

The Communities we will be working on (as observed from data): african', 'jewish', 'islam', 'women', 'arab', 'homosexual'

<a id='section01'></a>
### Importing Python Libraries and preparing the environment

In [1]:
# Installing the transformers library and additional libraries if looking process 

!pip install -q transformers

     |████████████████████████████████| 2.8 MB 11.0 MB/s 
     |████████████████████████████████| 636 kB 42.0 MB/s 
     |████████████████████████████████| 895 kB 45.6 MB/s 
     |████████████████████████████████| 52 kB 1.7 MB/s 
     |████████████████████████████████| 3.3 MB 43.0 MB/s 


In [2]:
# Importing stock ml libraries
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig

In [3]:
# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

<a id='section02'></a>
### Importing and Pre-Processing the domain data

We have used the data introduced in the [Hate Explain paper](https://arxiv.org/pdf/2012.10289.pdf).

Since our project is only focussed on the hatespeech, we preprocessed the data to take the subset of only the hate examples.

In [39]:
!git clone https://github.com/hate-alert/HateXplain.git

Cloning into 'HateXplain'...
remote: Enumerating objects: 386, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 386 (delta 73), reused 49 (delta 21), pack-reused 250
Receiving objects: 100% (386/386), 4.81 MiB | 13.20 MiB/s, done.
Resolving deltas: 100% (214/214), done.


In [43]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 1.6 MB 12.3 MB/s 
     |████████████████████████████████| 816 kB 38.9 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


     |████████████████████████████████| 5.8 MB 10.9 MB/s 
     |████████████████████████████████| 10.1 MB 40.2 MB/s 
     |████████████████████████████████| 623 kB 39.4 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 456 kB 37.1 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 13.6 MB 11.7 MB/s 
  Attempting uninstall: en-core-web-sm
    F

In [44]:
import os
import json
import spacy

nlp = spacy.load("en_core_web_sm")

# Carefully set the path of the dataset.json file
base_pth = os.getcwd()

new_data_pth = os.path.join(base_pth, "HateXplain", "Data", "dataset.json")

with open(new_data_pth, "r") as f:
    new_data_json = json.load(f)

# hate_twts = [i.strip() for i in hate_twts]    # Remove the extra spaces
# hate_twt_clnd = [i.strip() if ':' not in i else ':'.join(i.split(':')[1:]).strip() for i in hate_twts]

sents = []
targets = []
hate_terms = []
ents = []
poss = []
cnt_pos = []

for _, post in new_data_json.items():
    sent_targets = []
    sent_hate_terms = []
    label_cnt = 0
    sent_ents = []
    sent_cnt_pos = Counter()
    for annotation in post['annotators']:
        if annotation['label'] == 'hatespeech':
            label_cnt += 1
            sent_targets.extend(annotation['target'])
    if label_cnt >= 2:
        sent = " ".join(post['post_tokens'])
        sents.append(sent)
        targets.append(", ".join(list(set(sent_targets))))
        for rationale in post['rationales']:
            for idx, (annotator_rationale, sent_token) in enumerate(zip(rationale, post['post_tokens'])):
                if annotator_rationale == 1:
                    try:
                        sent_hate_terms.append(sent_token)
                    except Exception as e:
                        print("Error: " + str(e), idx)
                        break
        hate_terms.append(", ".join(list(set(sent_hate_terms))))
        
        # Get the entities
        doc = nlp(sent)
        for ent in doc.ents:
            sent_ents.append((ent.text, ent.label_))
        for token in doc:
            sent_cnt_pos[token.pos_] += 1
            
        ents.append(sent_ents)
        cnt_pos.append(dict(sent_cnt_pos))

print(sents)
print(targets)
print(hate_terms)
print(ents)
print(cnt_pos)

['<user> i am bit confused coz chinese ppl can not access twitter thn how this ching chong using it i think he pakistani 🤔 🤔 🤔', 'this bitch in whataburger eating a burger with no top bun and holding it palm up i hate white bitches', 'laura loomer raped me while screaming at me in her disgusting kike language and said we must exterminate the goyim laura loomer loomergate', 'and this is why i end up with nigger trainee doctors who can not speak properly lack basic knowledge of biology it truly scary if the public only knew', 'guilty until proven innocent unless you are a jew a nigger or a lover of both', 'tired i can not support abortion from a moral standpoint wired i can support it morally easy stop looking at those little niglets as future doctors and see them for what they really are future rapists of white women and future welfare recipient of white tax dollars and future democrat voters', '<number> <number> <percent> of brits think multiculturalism is a failure i guess the other <

### Store the output as a CSV

In [47]:
out_df = pd.DataFrame({'Post': sents, 'Target community': targets, 'Hate terms': hate_terms, 'Entities': ents, 'PoS':cnt_pos})
out_df.to_csv('data_analysis_1.csv', index=False)

## Preprocess the data to make it suitable for multilabel classification

In [15]:
from collections import Counter

fdf = pd.read_csv('data_analysis.csv')
trgts = fdf['Target community'].tolist()
trgt_list = []
for trgt in trgts:
    trgt = trgt.lower()
    trgt_list.extend([i.strip() for i in trgt.split(',')])

trgt_cnt = Counter()
for trgt in trgt_list:
    trgt_cnt[trgt] += 1
print(trgt_cnt)

feasible_comms = ['african', 'jewish', 'islam', 'women', 'arab', 'homosexual']#, 'refugee'] # Taking the 6 most frequently occuring classes

afcn_tgs, jew_tgs, is_tgs, wo_tgs, ar_tgs, hom_tgs = [0]*len(trgts), [0]*len(trgts), [0]*len(trgts), [0]*len(trgts), [0]*len(trgts), [0]*len(trgts)

for idx, trgt in enumerate(trgts):
    trgt = trgt.lower()
    for comm in trgt.split(','):
        comm = comm.strip()
        if comm == 'african':
            afcn_tgs[idx] = 1
        if comm == 'jewish':
            jew_tgs[idx] = 1
        if comm == 'islam':
            is_tgs[idx] = 1
        if comm == 'women':
            wo_tgs[idx] = 1
        if comm == 'arab':
            ar_tgs[idx] = 1
        if comm == 'homosexual':
            hom_tgs[idx] = 1

fdf['african'] = afcn_tgs
fdf['jewish'] = jew_tgs
fdf['islam'] = is_tgs
fdf['women'] = wo_tgs
fdf['arab'] = ar_tgs
fdf['homosexual'] = hom_tgs

fdf['feasible_comm'] = fdf['african']+fdf['jewish']+fdf['islam']+fdf['women']+fdf['arab']+fdf['homosexual']
fdf = fdf[fdf['feasible_comm'].apply(lambda x: x>0)]
fdf['one_hot_labels'] = list(fdf[feasible_comms].values)

fdf = fdf.rename(columns={"Post": "comment_text", "one_hot_labels": "list"})

fdf.head()

Counter({'african': 2524, 'jewish': 1575, 'islam': 1540, 'women': 964, 'arab': 945, 'homosexual': 870, 'other': 727, 'refugee': 428, 'caucasian': 377, 'men': 372, 'hispanic': 356, 'asian': 270, 'minority': 137, 'none': 105, 'nonreligious': 61, 'christian': 60, 'disability': 51, 'heterosexual': 50, 'indian': 40, 'indigenous': 35, 'hindu': 17, 'economic': 15, 'asexual': 5, 'bisexual': 2, 'buddhism': 1})


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,comment_text,Target community,Hate terms,Entities,PoS,african,jewish,islam,women,arab,homosexual,feasible_comm,list
1,this bitch in whataburger eating a burger with...,"Caucasian, Women","bitch, bitches, i, hate, white",[],"{'DET': 3, 'NOUN': 5, 'ADP': 3, 'VERB': 4, 'AD...",0,0,0,1,0,0,1,"[0, 0, 0, 1, 0, 0]"
2,laura loomer raped me while screaming at me in...,Jewish,"disgusting, language, exterminate, the, kike, ...","[('laura', 'PERSON')]","{'PROPN': 4, 'VERB': 5, 'PRON': 4, 'SCONJ': 1,...",0,1,0,0,0,0,1,"[0, 1, 0, 0, 0, 0]"
3,and this is why i end up with nigger trainee d...,African,"biology, properly, basic, trainee, nigger, not...",[],"{'CCONJ': 1, 'DET': 2, 'AUX': 2, 'ADV': 4, 'PR...",1,0,0,0,0,0,1,"[1, 0, 0, 0, 0, 0]"
4,guilty until proven innocent unless you are a ...,"African, Jewish","are, unless, nigger, you, a, of, jew, or, both...",[],"{'ADJ': 2, 'ADP': 2, 'VERB': 2, 'SCONJ': 1, 'P...",1,1,0,0,0,0,2,"[1, 1, 0, 0, 0, 0]"
5,tired i can not support abortion from a moral ...,African,"women, welfare, democrat, and, those, tax, rap...","[('white tax dollars', 'ORG'), ('democrat', 'N...","{'ADJ': 9, 'PRON': 6, 'AUX': 2, 'PART': 1, 'VE...",1,0,0,0,0,0,1,"[1, 0, 0, 0, 0, 0]"


In [5]:
# df = pd.read_csv("./data/train.csv")
# df['list'] = df[df.columns[2:]].values.tolist()
new_df = fdf[['comment_text', 'list']].copy()
new_df.head()

,comment_text,list
1,this bitch in whataburger eating a burger with...,"[0, 0, 0, 1, 0, 0]"
2,laura loomer raped me while screaming at me in...,"[0, 1, 0, 0, 0, 0]"
3,and this is why i end up with nigger trainee d...,"[1, 0, 0, 0, 0, 0]"
4,guilty until proven innocent unless you are a ...,"[1, 1, 0, 0, 0, 0]"
5,tired i can not support abortion from a moral ...,"[1, 0, 0, 0, 0, 0]"


<a id='section03'></a>
### Preparing the Dataset and Dataloader

In [6]:
# Sections of config

# Defining some key variables that will be used later on in the training
MAX_LEN = 200
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 10
LEARNING_RATE = 1e-05
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [7]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.comment_text = dataframe.comment_text
        self.targets = self.data.list
        self.max_len = max_len

    def __len__(self):
        return len(self.comment_text)

    def __getitem__(self, index):
        comment_text = str(self.comment_text[index])
        comment_text = " ".join(comment_text.split())

        inputs = self.tokenizer.encode_plus(
            comment_text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [8]:
# Creating the dataset and dataloader for the neural network

train_size = 0.8
train_dataset=new_df.sample(frac=train_size,random_state=200)
test_dataset=new_df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (5509, 2)
TRAIN Dataset: (4407, 2)
TEST Dataset: (1102, 2)


In [9]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

<a id='section04'></a>
### Creating the Neural Network for Fine Tuning 

In [10]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased', return_dict=False)
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 6)
    
    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

model = BERTClass()
model.to(device)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    

In [11]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [12]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

<a id='section05'></a>
### Fine Tuning the Model


In [13]:
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [14]:
for epoch in range(EPOCHS):
    train(epoch)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch: 0, Loss:  0.6587927341461182
Epoch: 1, Loss:  0.20538441836833954
Epoch: 2, Loss:  0.23798868060112
Epoch: 3, Loss:  0.14360511302947998
Epoch: 4, Loss:  0.1176178902387619
Epoch: 5, Loss:  0.055817611515522
Epoch: 6, Loss:  0.052830807864665985
Epoch: 7, Loss:  0.04674667865037918
Epoch: 8, Loss:  0.018026236444711685
Epoch: 9, Loss:  0.10467825829982758


<a id='section06'></a>
### Validating the Model

During the validation stage we pass the unseen data(Testing Dataset) to the model. This step determines how good the model performs on the unseen data. 

This unseen data is the 20% of `train.csv` which was seperated during the Dataset creation stage. 
During the validation stage the weights of the model are not updated. Only the final output is compared to the actual value. This comparison is then used to calcuate the accuracy of the model. 

As defined above to get a measure of our models performance we are using the following metrics. 
- Accuracy Score
- F1 Micro
- F1 Macro

We are getting amazing results for all these 3 categories just by training the model for 1 Epoch.

In [48]:
def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [33]:
for epoch in range(EPOCHS):
    outputs, targets = validation(epoch)
    outputs = np.array(outputs) >= 0.5
    accuracy = metrics.accuracy_score(targets, outputs)
    f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
    f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
    print(f"Accuracy Score = {accuracy}")
    print(f"F1 Score (Micro) = {f1_score_micro}")
    print(f"F1 Score (Macro) = {f1_score_macro}")

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Accuracy Score = 0.705989110707804
F1 Score (Micro) = 0.8811733014067644
F1 Score (Macro) = 0.8535108805347585
Accuracy Score = 0.705989110707804
F1 Score (Micro) = 0.8811733014067644
F1 Score (Macro) = 0.8535108805347585
Accuracy Score = 0.705989110707804
F1 Score (Micro) = 0.8811733014067644
F1 Score (Macro) = 0.8535108805347585
Accuracy Score = 0.705989110707804
F1 Score (Micro) = 0.8811733014067644
F1 Score (Macro) = 0.8535108805347585
Accuracy Score = 0.705989110707804
F1 Score (Micro) = 0.8811733014067644
F1 Score (Macro) = 0.8535108805347585
Accuracy Score = 0.705989110707804
F1 Score (Micro) = 0.8811733014067644
F1 Score (Macro) = 0.8535108805347585
Accuracy Score = 0.705989110707804
F1 Score (Micro) = 0.8811733014067644
F1 Score (Macro) = 0.8535108805347585
Accuracy Score = 0.705989110707804
F1 Score (Micro) = 0.8811733014067644
F1 Score (Macro) = 0.8535108805347585
Accuracy Score = 0.705989110707804
F1 Score (Micro) = 0.8811733014067644
F1 Score (Macro) = 0.8535108805347585
A

<a id='section07'></a>
### Saving the Trained Model

In [26]:
torch.save({
            'epoch': 10,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, '/content/drive/My Drive/model_1.pt')

## Future Work

- Visualization
- BiDAF like model to take cues from the target community identification
- We can try other fine tuned models

### References:
[Boiler plate code](https://github.com/abhimishra91/transformers-tutorials)

In [32]:
!pwd

/content


In [24]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
